### LangGraph Tutorial

this example shows codes for a simple chat graph

![](imgs/simple_chat_graph.png)

Reference:
* https://www.datacamp.com/tutorial/langgraph-tutorial

In [ ]:
# 1. Define the State Graph
from IPython.display import Image, display
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages

class State(TypedDict):
    # messages have the type "list".
    # The add_messages function appends messages to the list, rather than overwriting them
    messages: Annotated[list, add_messages]
graph_builder = StateGraph(State)

In [ ]:
import os
# get the parent directory of the current file
# current_path = os.path.dirname(__file__)
current_path = os.getcwd()
env_file_path = os.path.join(current_path, "envs", "azure.env")

In [ ]:
# 2. Initialize an LLM and add it as a Chatbot node
# https://python.langchain.com/docs/integrations/chat/

from langchain_openai import AzureChatOpenAI
from dotenv import dotenv_values

# Load the environment variables from the .env file
config = {**dotenv_values(env_file_path)}

llm = AzureChatOpenAI(
    azure_endpoint=config["AZURE_OPENAI_ENDPOINT"],
    openai_api_version=config["AZURE_OPENAI_API_VERSION"],
    azure_deployment=config["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"],
    api_key=config["AZURE_OPENAI_API_KEY"],
)

def chatbot(state: State):
    return {"messages": [llm.invoke(state["messages"])]}

'''
The first argument is the unique node name
The second argument is the function or object that will be called whenever the node is used.'''
graph_builder.add_node("chatbot", chatbot)

In [ ]:
# 3. Set edges

# Set entry and finish points
graph_builder.set_entry_point("chatbot")
graph_builder.set_finish_point("chatbot")

In [ ]:
# 4. Compile and Visualize the Graph

graph = graph_builder.compile()
print(type(graph))
try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    pass

In [ ]:
from typing import Generator

# Flag to control the streaming process
streaming = {"stop": False}

def stop_streaming(b):
    streaming["stop"] = True

def stream_graph(ai_graph, user_input: str)-> Generator[str, None, None]:
    # stream the token from the graph
    if user_input.lower() in ["quit", "exit", "q"]:
        # stream a fixed output
        yield "Goodbye!"
        yield "Exiting the program."
        stop_streaming(None)
    else:
        yield f"User: {user_input}"
        for event in ai_graph.stream({"messages": [("user", user_input)]}):
            for value in event.values():
                yield f"Assistant: {value['messages'][-1].content}"

In [ ]:
# 5. Create a Text Box for User Input and Output

import ipywidgets as widgets
from IPython.display import display, clear_output

# create a text box for user input with a send button
user_input = widgets.Text(
    value='',
    placeholder='Type your message here and enter...',
    description='User input:',
    disabled=False,
    layout=widgets.Layout(width='100%', height='40px', style={'description_width': 'initial'}),
)

# replace the widgets.Textarea with a text box for user output on the widget output area
output_text = widgets.Output(
    layout=widgets.Layout(
        width='100%',
        height='300px',
        overflow='auto',  # Ensure scrollbars appear if content overflows
        # style={'description_width': 'initial', 'white-space': 'pre-wrap'},  # Enable text wrapping
        style={'description_width': 'initial', 'white-space': 'pre-wrap'},  # Enable text wrapping
    ),
)


# Flag to control the streaming process
streaming = {"stop": False}

# function to handle user input and update the output
def handle_user_input(change):
    if change['name'] == 'value' and change['type'] == 'change' and change['new'] != '':
        # clear the output area
        # clear_output(wait=True)
        # get the user input
        user_input_value = change['new']
        # clear the user input text box
        user_input.value = ''

        # Reset the stop flag
        # streaming["stop"] = False

        # stream the graph with the user input
        output_stream = stream_graph(graph, user_input_value)
        
        # iterate over the output stream and update the output text box with the new value
        with output_text:
            for value in output_stream:
                if streaming["stop"]:
                    print("Streaming stopped.")
                    break
                print(value)
            print("\n")  # Add a newline for better readability


# observe changes in the user_input widget
user_input.continuous_update = False  # Disable continuous updates to avoid multiple triggers
user_input.observe(handle_user_input, names='value')


# Deprecate the on_submit and replace it with the observe method
# def on_send_button_click(b):
#     # clear the output area
#     clear_output(wait=True)
#     # get the user input
#     user_input_value = user_input.value
#     # clear the user input text box
#     user_input.value = ''

#     # stream the graph with the user input
#     output_stream = stream_graph(graph, user_input_value)
    
#     # iterate over the output stream and update the output text box with the new value
#     with output_text:
#         # clear the output text box
#         # output_text.clear_output(wait=True)
        
#         for value in output_stream:
#             print(value)
#         print("\n")  # Add a newline for better readability
# user_input.on_submit(on_send_button_click)


# display the text box and output area
display(user_input)
display(output_text)